# Memory Update Protocol of SHE
**Timothy Huang (behuangyi@gmail.com)**

* **Functions Defined**

In [1]:
from Crypto.Cipher import AES


# Miyaguchi-Preneel Compression Function
def AES_MP_Compression(padded_msgs):
    use_key_value = 0x00
    use_key_byte = use_key_value.to_bytes(16, byteorder='big')
    
    for m in padded_msgs:
        cipher = AES.new(use_key_byte, AES.MODE_ECB)
        
        msg_byte = m.to_bytes(16, byteorder='big')
        cipher_byte = cipher.encrypt(msg_byte)
        
        use_key_value = int(cipher_byte.hex(), 16) ^ use_key_value ^ m
        use_key_byte = use_key_value.to_bytes(16, byteorder='big')
    
    return use_key_byte.hex()


# KEY is a key string, and MESSAGES is a list contains message strings
def AES_CMAC(KEY, MESSAGE):
    MASK_1bit_ONE = 0x01
    MASK_128bit_ONE = int('f'*32, 16)
    CONST_R128 = 0x87
    CONST_ZERO = 0x00
    
    # Step1: Generate sub-keys
    key_byte = bytes.fromhex(KEY)
    cipher = AES.new(key_byte, AES.MODE_ECB)

    cipher_byte = cipher.encrypt(CONST_ZERO.to_bytes(16, byteorder='big'))
    CIPHERTEXT = cipher_byte.hex()
    CIPHER_L = int(CIPHERTEXT, 16)
    CIPHER_L_MSB = (CIPHER_L >> 127) & MASK_1bit_ONE

    if CIPHER_L_MSB == 0:
        K1_MAC = CIPHER_L << 1
    else:
        K1_MAC = (CIPHER_L << 1) ^ CONST_R128

    K1_MSB = (K1_MAC >> 127) & MASK_1bit_ONE

    if K1_MSB == 0:
        K2_MAC = K1_MAC << 1
    else:
        K2_MAC = (K1_MAC << 1) ^ CONST_R128

    K1_OUT = (K1_MAC & MASK_128bit_ONE).to_bytes(16, byteorder='big').hex()
    K2_OUT = (K2_MAC & MASK_128bit_ONE).to_bytes(16, byteorder='big').hex()

    # Step2: Start computing MAC using the sub-keys
    MESSAGES = []
    for idx, msg in enumerate(MESSAGE):
        if len(msg) != 0:
            MESSAGES.append(int(msg, 16))
        else:
            MESSAGES.append(0)

    mac_output = b'\x00'
    for idx in range(len(MESSAGES)-1):
        if idx == 0:    
            mac_output = cipher.encrypt(MESSAGES[idx].to_bytes(16, byteorder='big'))
        else:
            mac_input = int(mac_output.hex(), 16) ^ MESSAGES[idx]
            mac_output = cipher.encrypt(mac_input.to_bytes(16, byteorder='big'))

    msg_bit_len = len(MESSAGE[-1]) * 4

    if msg_bit_len < 128:
        padding_num = 128 - msg_bit_len - 1
        LAST_MSG_BIN = bin(MESSAGES[-1])[2:] + '1' + '0' * padding_num
        LAST_MSG = int(LAST_MSG_BIN, 2)
        mac_input = int(mac_output.hex(), 16) ^ LAST_MSG ^ int(K2_OUT, 16)
    else:
        mac_input = int(mac_output.hex(), 16) ^ MESSAGES[-1] ^ int(K1_OUT, 16)

    mac_output = cipher.encrypt(mac_input.to_bytes(16, byteorder='big'))

    return mac_output.hex()


# Input is a Hex string and format output the Hex value
def M_Value_Print(m_value_str):
    counter = 0
    for m_hex in m_value_str:
        counter += 1
        print(m_hex.upper(), end='')
        if counter == 2:
            counter = 0
            print(' ', end='')
    print()

* **Constant Value from SHE**

In [2]:
KEY_UPDATE_ENC_C = 0x010153484500800000000000000000b0
KEY_UPDATE_MAC_C = 0x010253484500800000000000000000b0

* **Input Value Pending Change**

In [3]:
# Default UID Value
UID_SHE_MODULE = 0x000000000000000000000000000001

# The secret used to do the update at AuthID
KEY_AUTH_ID = 0x000102030405060708090a0b0c0d0e0f

# The new key value expect to update to ID_KEY_SLOT
KEY_ID_NEW = 0x0f0e0d0c0b0a09080706050403020100

# Slot of the secret used for update key, value length 4 bits
# 0x1 is the MASTER_ECU_KEY slot
AuthID = 0x1

# Key slot to be updated, value length 4 bits
ID_KEY_SLOT = 0x4

# New Counter Value
CID = 0x0000001

WRITE_PROTECTION = '0'
BOOT_PROTECTION = '0'
DEBUGGER_PROTECTION = '0'
KEY_USAGE = '0'
WILDCARD = '0'

FID = WRITE_PROTECTION + BOOT_PROTECTION + DEBUGGER_PROTECTION + KEY_USAGE + WILDCARD

* **Key Derivation**

In [4]:
# Use the Secret to generate K1 and K2
K1 = AES_MP_Compression([KEY_AUTH_ID, KEY_UPDATE_ENC_C])
K2 = AES_MP_Compression([KEY_AUTH_ID, KEY_UPDATE_MAC_C])

# Use the New Key to generate K3 and K4
K3 = AES_MP_Compression([KEY_ID_NEW, KEY_UPDATE_ENC_C])
K4 = AES_MP_Compression([KEY_ID_NEW, KEY_UPDATE_MAC_C])

* **Calculate M1**

In [5]:
# The UID is specified to 120 bits
UID_BIT_LEN = 120
uid_bytes = UID_SHE_MODULE.to_bytes(UID_BIT_LEN//8, byteorder='big')

# M1 is the concatenation of the UID, the ID of Key to be updated, and the AuthID
M1 = uid_bytes.hex() + str(ID_KEY_SLOT) + str(AuthID)

* **Calculate M2**

In [6]:
# M2 is the CBC Encryption of INPUT_INFO with K1 and IV
# INPUT_INFO includes CID, FID and KEY_ID_NEW
FID_BIT_LEN = 5
CID_BIT_LEN = 28
CID_BIN_STR = bin(CID)[2:].zfill(CID_BIT_LEN)
FILL_GAP_ZEROS = '0' * (128 - CID_BIT_LEN - FID_BIT_LEN)

IV = 0x0
iv_byte = IV.to_bytes(16, byteorder='big')
key_byte = bytes.fromhex(K1)

cipher_byte = b''
INPUT_INFO = [int(CID_BIN_STR + FID + FILL_GAP_ZEROS, 2), KEY_ID_NEW]

for c in INPUT_INFO:
    cipher_byte += c.to_bytes(16, byteorder='big')

cipher = AES.new(key_byte, AES.MODE_CBC, iv_byte)
M2_byte = cipher.encrypt(cipher_byte)
M2 = M2_byte.hex()

* **Calculate M3**

In [7]:
from textwrap import wrap

# M3 is the MAC of [M1, M2] using K2
MESSAGE_IN = [M1] + wrap(M2, 32)
M3 = AES_CMAC(K2, MESSAGE_IN)

* **Calculate M4**

In [8]:
# M4 is consist by first 16 bytes same as M1
# and last 16 bytes the ECB Encryption of CID using K3
CID_PAD = '1' + (128 - 1- CID_BIT_LEN) * '0'
m4_star_info = int(CID_BIN_STR + CID_PAD, 2)
m4_star_byte = m4_star_info.to_bytes(16, byteorder='big')

cipher = AES.new(bytes.fromhex(K3), AES.MODE_ECB)
m4_cipher_byte = cipher.encrypt(m4_star_byte)
M4_STAR = m4_cipher_byte.hex()

M4_BLOCK1 = M1
M4 = [M4_BLOCK1, M4_STAR]

* **Calculate M5**

In [9]:
# M5 is the MAC of M4 using K4
M5 = AES_CMAC(K4, M4)

* **Format Print M1 to M5**

In [10]:
print("[M1 to M3]")
M_Value_Print(M1)
M_Value_Print(M2)
M_Value_Print(M3)
print()

print("[M4 to M5]")
print(M4[0], M4[1])
print(M5)

[M1 to M3]
00 00 00 00 00 00 00 00 00 00 00 00 00 00 01 41 
2B 11 1E 2D 93 F4 86 56 6B CB BA 1D 7F 7A 97 97 C9 46 43 B0 50 FC 5D 4D 7D E1 4C FF 68 22 03 C3 
B9 D7 45 E5 AC E7 D4 18 60 BC 63 C2 B9 F5 BB 46 

[M4 to M5]
00000000000000000000000000000141 b472e8d8727d70d57295e74849a27917
820d8d95dc11b4668878160cb2a4e23e


* **Collective Verification for Multiple Slaves**

In [11]:
# Random number with the length of 128 bits
KZKVERI_A = '0102030405060708090a0b0c0d0e0f00'

In [12]:
VERIFI_KEY = KEY_ID_NEW.to_bytes(16, byteorder='big').hex()

# MAC of the Random unmber with the new udpated key
KZKVERI_B = AES_CMAC(VERIFI_KEY, [KZKVERI_A])

print(VERIFI_KEY)
M_Value_Print(KZKVERI_A)
M_Value_Print(KZKVERI_B)

0f0e0d0c0b0a09080706050403020100
01 02 03 04 05 06 07 08 09 0A 0B 0C 0D 0E 0F 00 
F6 90 F2 55 63 A7 65 41 37 20 6A 83 BA 6A A0 EF 
